# Trucks detection with COLAB and Neptune

trying to follow https://neptune.ai/blog/how-to-train-your-own-object-detector-using-tensorflow-object-detection-api with my data


## Imports

In [1]:
import os

In [2]:
%cd models/research

/home/ec2-user/SageMaker/models/research


In [3]:
!python -m pip install --use-deprecated=legacy-resolver . 
# !python -m pip install --use-feature=2020-resolver .

Processing /home/ec2-user/SageMaker/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 352 kB 59.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 56.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 65.4 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 233 kB 69.6 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 70.9 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 6.3 MB/s  eta 0:00:01
  

     |████████████████████████████████| 99 kB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 213 kB 68.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 37.1 MB 30.8 MB/s eta 0:00:01     |████████████▍                   | 14.4 MB 30.8 MB/s eta 0:00:01     |███████████████▉                | 18.4 MB 30.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 55.2 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 155 kB 71.9 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 154 kB 74.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 54.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.9 MB/s  eta 0:00:01
     |███████████████████████████████

     |████████████████████████████████| 4.9 MB 53.4 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 60.1 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 72.3 MB/s eta 0:00:01
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1668658 sha256=2909c4170e160fd5edc51237885a8d0e0f46e566da7b37f23baed31a3d1a8642
  Stored in directory: /tmp/pip-ephem-wheel-cache-_tz6epyb/wheels/0f/f9/3c/80b1718c9d2eb5b6a4cae1495bfab9871f89c411c41b586424
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44009 sha256=71194d57b24edd395055e82e67f8abf235549bca2b269d477e56311629f227fb
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5b/6f/ca/80164a844e4691238e34a8b139b4695354df4490db91be61f7
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp36-cp36m-linux_x86_64.whl size=301742 sha256=fcdbd66bf2af498d3837369235807ec635e1b77b2e7

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## TF Lite

https://github.com/tensorflow/models/issues/7918
export_tflite_ssd_graph

 \
  --config_override=" \
    model{ \
      ssd { \
        image_resizer { \
          fixed_shape_resizer { \
            height: 640 \
            width: 640 \
          } \
        } \
      } \
    }"

In [6]:
%cd /home/ec2-user/SageMaker/

/home/ec2-user/SageMaker


In [4]:
! aws s3 ls s3://sagemaker-s3-luk/events/

                           PRE tf2-object-detection-2021-10-10-12-58-30-453/
                           PRE tf2-object-detection-2021-10-10-15-15-11-143/
                           PRE tf2-object-detection-2021-10-11-13-39-24-777/
                           PRE tf2-object-detection-2021-10-11-14-50-36-694/
                           PRE tf2-object-detection-2021-10-11-15-25-08-363/


In [5]:
%%bash
rm -rf /tmp/checkpoint/*
artifacts_path=s3://sagemaker-s3-luk/events/tf2-object-detection-2021-10-11-15-25-08-363/tensorboard-output
aws s3 sync $artifacts_path /tmp/checkpoint

download: s3://sagemaker-s3-luk/events/tf2-object-detection-2021-10-11-15-25-08-363/tensorboard-output/checkpoint to ../../../tmp/checkpoint/checkpoint
download: s3://sagemaker-s3-luk/events/tf2-object-detection-2021-10-11-15-25-08-363/tensorboard-output/ckpt-1.index to ../../../tmp/checkpoint/ckpt-1.index
download: s3://sagemaker-s3-luk/events/tf2-object-detection-2021-10-11-15-25-08-363/tensorboard-output/ckpt-2.index to ../../../tmp/checkpoint/ckpt-2.index
download: s3://sagemaker-s3-luk/events/tf2-object-detection-2021-10-11-15-25-08-363/tensorboard-output/train/events.out.tfevents.1633966242.ip-10-0-70-142.eu-central-1.compute.internal.28.1548.v2 to ../../../tmp/checkpoint/train/events.out.tfevents.1633966242.ip-10-0-70-142.eu-central-1.compute.internal.28.1548.v2
download: s3://sagemaker-s3-luk/events/tf2-object-detection-2021-10-11-15-25-08-363/tensorboard-output/ckpt-1.data-00000-of-00001 to ../../../tmp/checkpoint/ckpt-1.data-00000-of-00001
download: s3://sagemaker-s3-luk/even

In [7]:
%ls /tmp/checkpoint

checkpoint                  ckpt-1.index                ckpt-2.index  train/
ckpt-1.data-00000-of-00001  ckpt-2.data-00000-of-00001  eval/


In [8]:
!ls pretrained_models

ssd_mobilenet_v2_320x320_coco17_tpu-8
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8


In [17]:
model_name="ssd_resnet50_v1_fpn_640x640_coco17_tpu-8"

In [5]:
! ls -al tflite

total 444352
drwxr-x---  2 ec2-user ec2-user      4096 Oct 12 13:19 .
drwxr-xr-x 15 ec2-user ec2-user      4096 Oct 12 13:18 ..
-rw-rw-r--  1 ec2-user ec2-user 202685460 Oct 11 16:26 avocado_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tflite
-rw-rw-r--  1 ec2-user ec2-user   4183312 Oct 11 13:13 mobilenet_2_quant_detect.tflite
-rw-rw-r--  1 ec2-user ec2-user  18460280 Oct 11 14:14 model_320_nofpn.tflite
-rw-rw-r--  1 ec2-user ec2-user  12108572 Oct 10 16:08 model_640.tflite
-rw-rw-r--  1 ec2-user ec2-user   1866640 Oct 11 13:09 object_detection_mobile_object_localizer_v1_1_default_1.tflite
-rw-rw-r--  1 ec2-user ec2-user  13005876 Oct 11 07:59 original_fpnlite_640.tflite
-rw-rw-r--  1 ec2-user ec2-user 202685460 Oct 11 16:31 tflite_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tflite


In [9]:
%%bash
model_name=ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8
# model_name=ssd_mobilenet_v2_320x320_coco17_tpu-8
# model_name=ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

python models/research/object_detection/export_tflite_graph_tf2.py \
  --pipeline_config_path pretrained_models/$model_name/pipeline.docker.confignew \
  --trained_checkpoint_dir /tmp/checkpoint \
  --output_directory export/tflite_$model_name

2021-10-11 16:23:46.631715: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/efa/lib:/opt/amazon/efa/lib:/opt/amazon/efa/lib64:/usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/opt/amazon/efa/lib64:/opt/amazon/openmpi/lib64:/usr/local/lib:/usr/lib:/lib:/usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/opt/amazon/efa/lib64:/opt/amazon/openmpi/lib64:/usr/local/lib:/usr/lib:/lib:
2021-10-11 16:23:46.631749: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-10-11 16:23:49.820498: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEV

Graph is exported. Now let's convert to TFlite

In [24]:
!ls export/

tflite_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8  tflite_trucks_mobilenet_640
tflite_trucks_mobilenet_320_nofpn


In [2]:
!pip install tf-nightly

  Using cached tf_nightly-2.7.0.dev20210806-cp36-cp36m-manylinux2010_x86_64.whl (442.6 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tb_nightly-2.6.0a20210806-py3-none-any.whl (5.5 MB)
  Using cached grpcio-1.41.0-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.9 MB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached absl_py-0.14.1-py3-none-any.whl (131 kB)
  Using cached libclang-11.1.0-py2.py3-none-manylinux1_x86_64.whl (12.8 MB)
  Using cached tf_estimator_nightly-2.7.0.dev2021092408-py2.py3-none-any.whl (463 kB)
  Using cached keras_nightly-2.7.0.dev2021100607-py2.py3-none-any.whl (1.3 MB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached google_auth_oauthli

In [12]:
import tensorflow as tf
import os
import glob
tf.__version__

'2.7.0-dev20210806'

In [2]:
!ls export

tflite_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8  tflite_trucks_mobilenet_640
tflite_trucks_mobilenet_320_nofpn


In [4]:
model_name="trucks_mobilenet_640"
tflite_name=f"avocado_{model_name}.tflite"
saved_model_dir = f"export/tflite_{model_name}/saved_model"
os.listdir(saved_model_dir)

['assets', 'variables', 'saved_model.pb']

In [5]:
tflite_name

'avocado_trucks_mobilenet_640.tflite'

In [6]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the model.
with open(f"tflite/{tflite_name}", 'wb') as f:
  f.write(tflite_model)

os.path.getsize(f"tflite/{tflite_name}")

12109192

In [7]:
! aws s3 sync ./tflite/ s3://sagemaker-s3-luk/output

upload: tflite/avocado_trucks_mobilenet_640.tflite to s3://sagemaker-s3-luk/output/avocado_trucks_mobilenet_640.tflite


In [11]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
filename = f'tflite/optdef_{tflite_name}'

with open(filename, 'wb') as f:
  f.write(tflite_model)

os.path.getsize(filename)

4217776

In [14]:
data_dir = "images/avocadoproject"
all_image_paths = [a for a in glob.glob(data_dir+"/*.JPEG")]
all_image_paths[:5]

['images/avocadoproject/IMG_1213.JPEG',
 'images/avocadoproject/IMG_1096.JPEG',
 'images/avocadoproject/IMG_1157.JPEG',
 'images/avocadoproject/IMG_1209.JPEG',
 'images/avocadoproject/IMG_1214.JPEG']

In [19]:
def representative_data_gen():
    for imagepath in all_image_paths:
        image = tf.io.read_file(imagepath)
        image = tf.compat.v1.image.decode_jpeg(image)
        image = preprocess(image)
        yield [image]

In [20]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()
filename = f'tflite/optdef_quant_{tflite_name}'

with open(filename, 'wb') as f:
  f.write(tflite_model)

os.path.getsize(filename)

NameError: name 'preprocess' is not defined